In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
local_dir = '/content/drive/MyDrive/Acid Analysis/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px

# 1. Load Data

In [ ]:
old_ref = pd.read_csv(local_dir + 'antibody reference.csv')
old_ref['Antibody Code'] = old_ref['Antibody Code'].astype(str).str.strip()
old_ref = old_ref[~old_ref.duplicated()]

In [ ]:
#Deduplicate 'Antibody Code'
old_ref['NaN_count'] = old_ref.isnull().sum(axis=1)
old_ref.sort_values('NaN_count').drop_duplicates(subset=['Antibody Code'], keep='first')
old_ref = old_ref.drop(columns=['NaN_count'])
old_ref

,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID
0,NN441-2,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN
1,NN441-3,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN
2,sc-133233,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN
3,sc-1657,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN
4,NN001,14-3-3 z,14-3-3 z,Pan-specific,Mouse monoclonal,14-3-3 protein zeta (cross-reacts with other i...,P63104,NP_003397
...,...,...,...,...,...,...,...,...
3795,PK933,NaN,GSK3b,T43,NaN,NaN,P49841,NaN
3796,PN887,NaN,MATR3,T130,NaN,NaN,P43243,NaN
3797,PN933,NaN,AKAP12,S696,NaN,NaN,Q02952,NaN
3798,PN967,NaN,LRP4,S1887,NaN,NaN,O75096,NaN


In [ ]:
new_data = pd.read_excel(local_dir + 'CombinedArrays.xlsx', sheet_name='data').iloc[1:][['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']]
new_data.columns = ['name', 'site', 'antibody']

In [ ]:
new_ref = new_data
new_ref = new_ref[~new_ref[['site', 'antibody']].duplicated()]
new_ref.loc[:, 'antibody'] = new_ref['antibody'].astype(str).str.strip()
new_ref

<ipython-input-6-e65802ee6446>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ref.loc[:, 'antibody'] = new_ref['antibody'].astype(str).str.strip()


,name,site,antibody
1,14-3-3-pan-1,Pan,NN441-2
3,14-3-3-pan-2,Pan,NN441-3
5,14-3-3-pan,Pan,sc-1657
7,14-3-3-B-1,Pan,NN452-1
9,14-3-3-B-2,Pan,NN452-2
...,...,...,...
4045,ZAP70,Y248,PK860
4047,ZAP70,Y292,PK861
4049,ZAP70,Y319,PK862
4051,ZAP70,Y492+pY493,PK863


# 2. Combine old and new

In [ ]:
both_merge = old_ref.merge(new_ref, left_on='Antibody Code', right_on='antibody', how='outer')

# Make all string, but keep Nans
both_merge = both_merge.applymap(lambda x: str(x) if not pd.isna(x) else x)

# Deduplicate identical rows
both_merge = both_merge[~both_merge.duplicated(keep=False)]
both_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3796
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Antibody Code      3796 non-null   object
 1   Antibody Name      411 non-null    object
 2   Target Name        3796 non-null   object
 3   Pan or P-Site      3519 non-null   object
 4   Antibody Type      409 non-null    object
 5   Target Full Name   1529 non-null   object
 6   Target Uniprot ID  3770 non-null   object
 7   Target Refseq ID   1489 non-null   object
 8   name               2030 non-null   object
 9   site               2030 non-null   object
 10  antibody           2030 non-null   object
dtypes: object(11)
memory usage: 356.0+ KB


In [ ]:
# Fix is to move p-site data from new data to old data.
both_merge.loc[both_merge['Pan or P-Site'].isna(), 'Pan or P-Site'] = both_merge.loc[both_merge['Pan or P-Site'].isna(), 'site']
both_merge.loc[both_merge['Antibody Code'].isna(), 'Antibody Code'] = both_merge.loc[both_merge['Antibody Code'].isna(), 'antibody']
both_merge.loc[both_merge['Target Name'].isna(), 'Target Name'] = both_merge.loc[both_merge['Target Name'].isna(), 'name']

# This is me keeping the P-Site normalisation simple
both_merge['Pan or P-Site'] = both_merge['Pan or P-Site'].replace({'pan': 'Pan-specific', 'Pan': 'Pan-specific'})
both_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3796
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Antibody Code      3797 non-null   object
 1   Antibody Name      411 non-null    object
 2   Target Name        3797 non-null   object
 3   Pan or P-Site      3790 non-null   object
 4   Antibody Type      409 non-null    object
 5   Target Full Name   1529 non-null   object
 6   Target Uniprot ID  3770 non-null   object
 7   Target Refseq ID   1489 non-null   object
 8   name               2030 non-null   object
 9   site               2030 non-null   object
 10  antibody           2030 non-null   object
dtypes: object(11)
memory usage: 356.0+ KB


In [ ]:
both_merge

,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID,name,site,antibody
0,NN441-2,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN,14-3-3-pan-1,Pan,NN441-2
1,NN441-3,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN,14-3-3-pan-2,Pan,NN441-3
2,sc-133233,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN,NaN,NaN,NaN
3,sc-1657,NaN,14-3-3 (KCIP-1),Pan-specific,NaN,NaN,P31946,NaN,14-3-3-pan,Pan,sc-1657
4,NN001,14-3-3 z,14-3-3 z,Pan-specific,Mouse monoclonal,14-3-3 protein zeta (cross-reacts with other i...,P63104,NP_003397,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3792,PN887,NaN,MATR3,T130,NaN,NaN,P43243,NaN,MATR3,T130,PN887
3793,PN933,NaN,AKAP12,S696,NaN,NaN,Q02952,NaN,AKAP12,S696,PN933
3794,PN967,NaN,LRP4,S1887,NaN,NaN,O75096,NaN,LRP4,S1887,PN967
3795,sc-6009,NaN,MuSK,Pan-specific,NaN,NaN,O15146,NaN,MuSK,Pan,sc-6009


## These are unsolved or semi-unsolved issues

In [ ]:
#Duplicated antibodies
both_merge['NaN_count'] = both_merge.isnull().sum(axis=1)
both_merge[both_merge['Antibody Code'].duplicated(keep=False)].sort_values(['Antibody Code', 'NaN_count'])

,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID,name,site,antibody,NaN_count
2112,AP7908a,NaN,MEKK2 (MAP3K2),Pan-specific,NaN,NaN,Q9Y2U5,NaN,MEKK2 N-term.,Pan,AP7908a,4
2113,AP7908a,NaN,MEKK2 N-term.,Pan-specific,NaN,NaN,Q9Y2U5,NaN,MEKK2 N-term.,Pan,AP7908a,4
3590,B50263,NaN,TPPb,Pan-specific,NaN,NaN,Q6XPS3,NaN,NaN,NaN,NaN,7
3591,B50263,NaN,TPPb,NaN,NaN,NaN,Q6XPS3,NaN,NaN,NaN,NaN,8
340,NK155-8,NaN,Raf1 (c-Raf; RafC),Pan-specific,NaN,NaN,P04049,NaN,BRaf-1 changed to Raf1-6,Pan,NK155-8,4
339,NK155-8,NaN,BRaf-1 changed to Raf1-6,Pan-specific,NaN,NaN,NaN,NaN,BRaf-1 changed to Raf1-6,Pan,NK155-8,5
95,PN932,NaN,AKAP12,S627+S629,NaN,NaN,Q02952,NaN,AKAP12,S627+S629,PN932,4
96,PN932,NaN,AKAP12,S13,NaN,NaN,Q02952,NaN,GFAP,S13,PN932,4
97,PN932,NaN,GFAP,S627+S629,NaN,NaN,P14136,NaN,AKAP12,S627+S629,PN932,4
98,PN932,NaN,GFAP,S13,NaN,NaN,P14136,NaN,GFAP,S13,PN932,4


In [ ]:
# Notice the different Uniprots, I'm not gonna deal with this now.
dupes = both_merge[both_merge['Antibody Code'].duplicated(keep=False)]
group_count = dupes.groupby('Antibody Code')['Target Uniprot ID'].nunique()
idx = group_count[group_count != 1].index
dupes[dupes['Antibody Code'].isin(idx)]

,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID,name,site,antibody,NaN_count
95,PN932,NaN,AKAP12,S627+S629,NaN,NaN,Q02952,NaN,AKAP12,S627+S629,PN932,4
96,PN932,NaN,AKAP12,S13,NaN,NaN,Q02952,NaN,GFAP,S13,PN932,4
97,PN932,NaN,GFAP,S627+S629,NaN,NaN,P14136,NaN,AKAP12,S627+S629,PN932,4
98,PN932,NaN,GFAP,S13,NaN,NaN,P14136,NaN,GFAP,S13,PN932,4
1431,sc-7291,NaN,GSK3a,Pan-specific,NaN,NaN,P49840,NaN,GSK-3β,Pan,sc-7291,4
1432,sc-7291,NaN,GSK-3?,Pan-specific,NaN,NaN,P49841,NaN,GSK-3β,Pan,sc-7291,4


In [ ]:
dupes = both_merge[both_merge['Antibody Code'].duplicated(keep=False)]
group_count = dupes.groupby('Antibody Code')['Pan or P-Site'].nunique()
idx = group_count[group_count != 1].index

display(dupes[dupes['Antibody Code'].isin(idx)])

# Make these disagreements into Pan
both_merge.loc[both_merge['Antibody Code'].isin(idx), 'Pan or P-Site'] = 'Pan-specific'

,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID,name,site,antibody,NaN_count
95,PN932,NaN,AKAP12,S627+S629,NaN,NaN,Q02952,NaN,AKAP12,S627+S629,PN932,4
96,PN932,NaN,AKAP12,S13,NaN,NaN,Q02952,NaN,GFAP,S13,PN932,4
97,PN932,NaN,GFAP,S627+S629,NaN,NaN,P14136,NaN,AKAP12,S627+S629,PN932,4
98,PN932,NaN,GFAP,S13,NaN,NaN,P14136,NaN,GFAP,S13,PN932,4


In [ ]:
# Ok now finally deduplicate by antibody
def reduce_antibody_group(G):
  top_row = G.sort_values('NaN_count').iloc[0]

  all_sites = pd.concat([G['Pan or P-Site'], G['site']])
  all_sites = all_sites[~all_sites.isna()]
  all_sites = "".join(list(all_sites))

  if 'pan' in all_sites.lower():
    top_row['Pan or P-Site'] = 'Pan-specific'
  return top_row

# Drop the duplicate antibody code rows, but if any of the sites are pan, make it pan.
reduced = both_merge.groupby('Antibody Code').apply(reduce_antibody_group).reset_index(drop=True)
print(reduced['Antibody Code'].duplicated().any())
reduced

False


,Antibody Code,Antibody Name,Target Name,Pan or P-Site,Antibody Type,Target Full Name,Target Uniprot ID,Target Refseq ID,name,site,antibody,NaN_count
0,05-0073 (Clone 8F7),NaN,FAK (PTK2),Y397,NaN,NaN,Q05397,NaN,NaN,NaN,NaN,7
1,05-184,NaN,Src,Pan-specific,NaN,NaN,P12931,NaN,Src,Pan,05-184,4
2,05-636,NaN,H2AFX (H2AX; Histone H2A.X),S139,NaN,NaN,P16104,NaN,NaN,NaN,NaN,7
3,05-746,NaN,Histone H3,T3,NaN,NaN,P84243,NaN,Histone H3,T3,05-746,4
4,06-0020,NaN,Akt1 (PKBa),NaN,NaN,NaN,P31749,NaN,NaN,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...
3785,sc-960,NaN,MKK3 (MAP2K3; MEK3),Pan-specific,NaN,NaN,P46734,NaN,MEK3,Pan,sc-960,4
3786,sc-964,NaN,MKK4 (MAP2K4; MEK4),Pan-specific,NaN,NaN,P45985,NaN,MEK4,Pan,sc-964,4
3787,sc-9898,NaN,Csk,Pan-specific,NaN,NaN,P41240,NaN,Csk,Pan,sc-9898,4
3788,sc-9968,NaN,CDKN2A (p16; p16-INK4a),Pan-specific,NaN,NaN,Q8N726,NaN,p16 (50.1),Pan,sc-9968,4


# Export the merged table

In [ ]:
reduced.drop(columns=['name', 'site', 'antibody', 'NaN_count']).to_csv(local_dir + 'antibody revised.csv', index=False)